In [ ]:
import requests
from kafka import KafkaProducer
import json
import time

In [ ]:
bootstrap_servers = 'kafka:9092'
topic_name = 'nba-API-data'
producer = KafkaProducer(bootstrap_servers=bootstrap_servers, value_serializer=lambda x: json.dumps(x).encode('utf-8'))

In [ ]:
base_url = 'https://www.balldontlie.io/api/v1'
seasons = [1995, 1996, 1997, 2005, 2006, 2007, 2011, 2012, 2013]

In [ ]:
def get_players():
    for player_id in range(100, 200):
        for season in seasons:
            player_details = {
                'id': player_id,
                'first_name': None,  # Placeholder for first name
                'last_name': None  # Placeholder for last name
            }
            
            retries = 0
            max_retries = 5
            
            while True:
                player_url = f'{base_url}/players/{player_id}'
                response = requests.get(player_url)
                
                if response.status_code == 200:
                    player_data = response.json()
                    if 'first_name' in player_data:
                        player_details['first_name'] = player_data['first_name']
                    if 'last_name' in player_data:
                        player_details['last_name'] = player_data['last_name']
                    break  # Break the while loop if the response code is 200
                else:
                    retries += 1
                    if retries > max_retries:
                        break  # Break the while loop if maximum retries reached
            
            yield player_details, season

            
def get_season_stats(player_id, season):
    url = f'{base_url}/season_averages?season={season}&player_ids[]={player_id}'
    response = requests.get(url)
    
    if response.status_code == 200:
        try:
            season_stats = response.json()
            if 'data' in season_stats and season_stats['data']:
                return season_stats['data']
        except json.JSONDecodeError:
            pass
    
    return None

In [ ]:
for player_details, season in get_players():
    player_id = player_details['id']
    player_season_stats = get_season_stats(player_id, season)

    if player_season_stats is not None:
        full_name = f"{player_details['first_name']} {player_details['last_name']}"

        player_data = {
            'full_name': full_name,
            'season': season,
            'season_stats': player_season_stats
        }
        producer.send(topic_name, value=player_data)
        
        print(f"Sent: {full_name} - {season}")

print("Loop exited")

In [ ]:
producer.flush()
producer.close()